# Code plagirism detector

Fermin Mendez A01703366
Adrian Matute
Alan Razo A01703350

## Import libraries

In [3]:
import pandas as pd
import os
import glob
import string
import numpy as np


## Load example data set

In [6]:
code_1="""
public class PalindromeChecker {
    public static boolean isPalindrome(String input) {
        String clean = input.replaceAll("\\s+", "").toLowerCase();
        String reversed = new StringBuilder(clean).reverse().toString();
        return clean.equals(reversed);
    }

    public static void main(String[] args) {
        String test = "A man a plan a canal Panama";
        if (isPalindrome(test)) {
            System.out.println("\"" + test + "\" is a palindrome.");
        } else {
            System.out.println("\"" + test + "\" is not a palindrome.");
        }
    }
}
"""

code_2="""
public class BubbleSortWithCount {
    public static void bubbleSort(int[] arr) {
        int swapCount = 0;
        for (int i = 0; i < arr.length - 1; i++) {
            boolean swapped = false;
            for (int j = 0; j < arr.length - i - 1; j++) {
                if (arr[j] > arr[j + 1]) {
                    // Swap elements
                    int temp = arr[j];
                    arr[j] = arr[j + 1];
                    arr[j + 1] = temp;
                    swapped = true;
                    swapCount++;
                }
            }
            if (!swapped) break;
        }
        System.out.println("Array sorted with " + swapCount + " swaps.");
    }

    public static void main(String[] args) {
        int[] data = {5, 1, 4, 2, 8};
        bubbleSort(data);
        for (int n : data) {
            System.out.print(n + " ");
        }
    }
}
"""

In [7]:
reserved_words_in_java = [
    "abstract", "assert", "boolean", "break", "byte", "case", "catch", "char",
    "class", "const", "continue", "default", "do", "double", "else", "enum",
    "extends", "final", "finally", "float", "for", "goto", "if", "implements",
    "import", "instanceof", "int", "interface", "long", "native", "new",
    "null", "package", "private", "protected", "public", "return",
    "short", "static", "strictfp", "super", "switch", "synchronized",
    "this", "throw", "throws", "transient", "try", "void",
    "volatile",  # Added 'volatile' to the list
    # 'while' is not included as it is not a reserved word in Java
]

## Auxiliar functions

In [ ]:
def clearCommentsinJavaCode(code):
    """
    This function removes comments from Java code.
    """
    # Remove single-line comments
    code = '\n'.join([line.split('//')[0] for line in code.split('\n')])
    
    # Remove multi-line comments
    while '/*' in code and '*/' in code:
        start = code.index('/*')
        end = code.index('*/', start) + 2
        code = code[:start] + code[end:]
    
    return code

## Approach 1. Cosine similarity

In [ ]:
def vectorizeJavaCode(code):
   
    